In [1]:
%run functions.ipynb

In [2]:
import geopandas as gpd
import pandas as pd
import os

In [6]:
# schema
col_names = {
    'Name': 'ParkRideName',
    'Description': 'HTMLMetaData',
    'geometry': 'geometry' # don't mess with this - core part of gpd geodataframe
}

# load stations dataframe
filepath = '..\\data\\bronze\\CTA_ParkRide.kml'
gdf = gpd.read_file(filepath).rename(columns=col_names)

print('Loaded', len(gdf), 'rows')
display(gdf.head())

Loaded 17 rows


,ParkRideName,HTMLMetaData,geometry
0,Cumberland,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.83747 41.98337 0)
1,Midway Airport,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.73935 41.78521 0)
2,Garfield-South Elevated,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.61875 41.79568 0)
3,Kimball,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.71251 41.96704 0)
4,54th/Cermak,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.75739 41.85178 0)


In [8]:
# extract metadata from the html column
metadata_items = [
    'Rail Station', 
    'Rail Line', 
    'ADA',
    'Spaces'
]

gdf['ExtractedMetaData'] = gdf.HTMLMetaData.apply(extract_metadata)

display(gdf.head())


,ParkRideName,HTMLMetaData,geometry,ExtractedMetaData
0,Cumberland,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.83747 41.98337 0),"[Cumberland, Blue Line, Yes, 1,633]"
1,Midway Airport,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.73935 41.78521 0),"[Midway Airport, Orange Line, Yes, 299]"
2,Garfield-South Elevated,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.61875 41.79568 0),"[Garfield-South Elevated, Green Line, Yes, 117]"
3,Kimball,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.71251 41.96704 0),"[Kimball, Brown Line, Yes, 73]"
4,54th/Cermak,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.75739 41.85178 0),"[54th/Cermak, Pink Line, Yes, 175]"


In [9]:
# explode the metadata out into columns
new_columns = ['StationName', 'RailLine', 'ADAAccessible', 'Spaces']

metadata_col = gdf.ExtractedMetaData
for i, col in enumerate(new_columns):
    gdf[col] = metadata_col.str[i]

display(gdf.head())


,ParkRideName,HTMLMetaData,geometry,ExtractedMetaData,StationName,RailLine,ADAAccessible,Spaces
0,Cumberland,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.83747 41.98337 0),"[Cumberland, Blue Line, Yes, 1,633]",Cumberland,Blue Line,Yes,"1,633"
1,Midway Airport,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.73935 41.78521 0),"[Midway Airport, Orange Line, Yes, 299]",Midway Airport,Orange Line,Yes,299
2,Garfield-South Elevated,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.61875 41.79568 0),"[Garfield-South Elevated, Green Line, Yes, 117]",Garfield-South Elevated,Green Line,Yes,117
3,Kimball,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.71251 41.96704 0),"[Kimball, Brown Line, Yes, 73]",Kimball,Brown Line,Yes,73
4,54th/Cermak,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.75739 41.85178 0),"[54th/Cermak, Pink Line, Yes, 175]",54th/Cermak,Pink Line,Yes,175


In [11]:
# convert spaces to an int type
gdf['Spaces'] = gdf.Spaces.str.replace(',', '').astype(int)

display(gdf.head())

,ParkRideName,HTMLMetaData,geometry,ExtractedMetaData,StationName,RailLine,ADAAccessible,Spaces
0,Cumberland,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.83747 41.98337 0),"[Cumberland, Blue Line, Yes, 1,633]",Cumberland,Blue Line,Yes,1633
1,Midway Airport,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.73935 41.78521 0),"[Midway Airport, Orange Line, Yes, 299]",Midway Airport,Orange Line,Yes,299
2,Garfield-South Elevated,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.61875 41.79568 0),"[Garfield-South Elevated, Green Line, Yes, 117]",Garfield-South Elevated,Green Line,Yes,117
3,Kimball,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.71251 41.96704 0),"[Kimball, Brown Line, Yes, 73]",Kimball,Brown Line,Yes,73
4,54th/Cermak,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",POINT Z (-87.75739 41.85178 0),"[54th/Cermak, Pink Line, Yes, 175]",54th/Cermak,Pink Line,Yes,175


In [14]:
# drop no longer needed columns
select_cols = [
    'ParkRideName',
    'geometry',
    'StationName',
    'RailLine',
    'ADAAccessible',
    'Spaces'
]
gdf = gdf[select_cols]

# save to file
filepath = '..\\data\\silver\\'
filename = 'DimParkRide.parquet'
gdf.to_parquet(filepath+filename)

# verify
os.listdir(filepath)

['DimParkRide.parquet', 'DimRailLine.parquet', 'DimRailStation.parquet']